# 光学シミュレーション

## 参考文献
吉田貞史：『薄膜・光デバイス』,東京大学出版会（1994）

羽渕仁恵：光学薄膜のシミュレーションソフトウェアの開発 岐阜高専紀要 39(2004) 47 

慈幸範洋ら：光学機能性フィルムのシミュレーションによる設計と実験的検証 神戸製鋼技報 65(2015)(9) 51

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from numpy import arcsin,sin,cos,sqrt,exp,pi,array,identity,dot
import enum
Pol = enum.Enum('Pol','s p sp')

## スネルの法則
\begin{equation*}
 n_{0}\sin\phi_{0} = n_{1}\sin\phi_{1}  \\
 \phi_{1} = \sin^{-1} \left(\frac{n_{0}}{n_{1}}\sin\phi_{0}\right)
\end{equation*}

In [3]:
def SnellLaw(n0,n1,angle):
    return arcsin(n0/n1*sin(angle))

## 実効屈折率
\begin{eqnarray*}
 \eta_{j} &=& -n_{j}\cos\phi_{j}  & :s-polarized\\
          &=&\frac{n_{j}}{\cos \phi_{j}} & :p-polarized
\end{eqnarray*}

In [4]:
def EffectiveRefractiveIndex(n,angle,pol):
    if(pol == Pol.s):
        return -n*cos(angle)
    else:
        return n/cos(angle)

## フレネル係数
\begin{equation*}
 r = \frac{\eta_{0}-\eta_{1}}{\eta_{0}+\eta_{1}} \\
 t = \sqrt{\frac{n_{0} \cos \phi_{0}}{n_{1} \cos \phi_{1}}}\cdot\frac{2\sqrt{\eta_{0}\eta_{1}}}{\eta_{0}+\eta_{1}}
\end{equation*}

In [5]:
def FresnelCoefficient(n0,n1,angle0,pol):
    eta0=EffectiveRefractiveIndex(n0,angle0,pol)
    angle1=SnellLaw(n0,n1,angle0)
    eta1=EffectiveRefractiveIndex(n1,angle1,pol)
    r = (eta0-eta1)/(eta0+eta1)
    t = sqrt(n0*cos(angle0)/(n1*cos(angle1)))*2*sqrt(eta0*eta1)/(eta0+eta1)
    return r,t 

## 位相差
\begin{equation*}
 2\delta = \frac{4\pi}{\lambda}nd\cos\phi
\end{equation*}

In [6]:
def PhaseDifference(n,d,angle,wl):
    return 2*pi*n*d*cos(angle)/wl

## 特性マトリクス
\begin{equation*}
M_{j} = 
        \left(
         \begin{array}{cc} 
          \cos \delta_{j} & i \eta_{j}^{-1} \sin \delta_{j} \\
          i \eta_{j} \sin \delta_{j}& \cos \delta_{j}\\
         \end{array}
         \right)
\end{equation*}

In [7]:
class Material:
    def __init__(self,n={550:1.5}):
        self.n = n
    def getIndex(self,wl):
        value = self.n.get(wl)
        if value == None:
            nl = self.n.keys()
            l = [x for x in nl if x < wl]
            h = [x for x in nl if x > wl]
            if len(l) != 0 and len(h) != 0:
                x1 = max(l)
                x2 = min(h)
                y1 = self.n.get(x1)
                y2 = self.n.get(x2)
                value = (y2-y1)/(x2-x1)*(wl-x1)+y1
            elif len(l) != 0:
                value = self.n.get(max(l))
            elif len(h) != 0:
                value = self.n.get(min(h))
            else:
                value = None
        return value

In [8]:
class Layer:
    def __init__(self,material,thickness):
        self.material = material
        self.thickness = thickness
    def getSpecificMatrix(self,n0,angle0,wl,pol):
        n = self.material.getIndex(wl)
        angle = SnellLaw(n0,n,angle0)
        delta = PhaseDifference(n,self.thickness,angle,wl)
        eta = EffectiveRefractiveIndex(n,angle,pol)
        return array([[cos(delta),1j/eta*sin(delta)],[1j*eta*sin(delta),cos(delta)]])

## 境界面での反射・透過
\begin{equation*}
 R=|r_{1}|^{2} \\
 T=\frac{n_{1} \cos \phi_{1}}{n_{0} \cos \phi_{0}}|t_{1}|^{2} 
\end{equation*}

In [9]:
def SemiInfinite(matIn,matOut,angle0,wl,pol):
    if pol == Pol.sp:
        rrs,tts = SemiInfinite(matIn,matOut,angle0,wl,Pol.s)
        rrp,ttp = SemiInfinite(matIn,matOut,angle0,wl,Pol.p)
        rr = (rrs+rrp)/2
        tt = (tts+ttp)/2
    else:
        n0 = matIn.getIndex(wl)
        n1 = matOut.getIndex(wl)
        angle1 = SnellLaw(n0,n1,angle0)
        r,t = FresnelCoefficient(n0,n1,angle0,pol)
        rr = abs(r)**2
        tt = n1*cos(angle1)/(n0*cos(angle0))*abs(t)**2
    return rr,tt

\begin{equation*}
 R=\frac{r_{1}+r_{2}\mathrm{e}^{-2i\delta_{1}}}{1+r_{1}r_{2}\mathrm{e}^{-2i\delta_{1}}} \\
 T=\frac{t_{1}t_{2}\mathrm{e}^{-i\delta_{1}}}{1+r_{1}r_{2}\mathrm{e}^{-2i\delta_{1}}}
\end{equation*}

    n0
------------
    n,t
------------
    n1

In [10]:
def Monolayer(matIn,matOut,layer,angle0,wl,pol):
    n0 = matIn.getIndex(wl)
    n1 = matOut.getIndex(wl)
    n = layer.material.getIndex(wl)
    thickness = layer.thickness
    angle = SnellLaw(n0,n,angle0)
    angle1 = SnellLaw(n0,n1,angle0)
    if pol == Pol.sp:
        rrs,tts = Monolayer(matIn,matOut,layer,angle0,wl,Pol.s)
        rrp,ttp = Monolayer(matIn,matOut,layer,angle0,wl,Pol.p)
        rr = (rrs+rrp)/2
        tt = (tts+ttp)/2
    else:
        r1,t1 = FresnelCoefficient(n0,n,angle0,pol)
        r2,t2 = FresnelCoefficient(n,n1,angle,pol)
        delta1 = PhaseDifference(n,thickness,angle,wl)
        r = (r1+r2*exp(-2j*delta1))/(1+r1*r2*exp(-2j*delta1))
        t = t1*t2*exp(-2j*delta1)/(1+r1*r2*exp(-2j*delta1))
        rr = abs(r)**2
        tt = n1*cos(angle1)/(n0*cos(angle0))*abs(t)**2
    return rr,tt

## 特性マトリクスを使った場合
\begin{equation*}
 R=\frac{\eta_{0}(m_{11}+\eta_{N+1}m_{12})-(m_{21}+\eta_{N+1}m_{22})}{\eta_{0}(m_{11}+\eta_{N+1}m_{12})+(m_{21}+\eta_{N+1}m_{22})} \\
 T=\gamma\frac{2\eta_{0}}{\eta_{0}(m_{11}+\eta_{N+1}m_{12})+(m_{21}+\eta_{N+1}m_{22})}
\end{equation*}

\begin{equation*}
\gamma = \left\{
 \begin{array}{l}
  1 & :s-polarized\\
  \frac{\cos\phi_{0}}{\cos\phi_{N+1}} & :p-polarized
 \end{array}
 \right.
\end{equation*}

In [11]:
def Multilayer(matIn,matOut,layers,angle0,wl,pol):
    n0 = matIn.getIndex(wl)
    n1 = matOut.getIndex(wl)
    angle1 = SnellLaw(n0,n1,angle0)
    eta0 = EffectiveRefractiveIndex(n0,angle0,pol)
    eta1 = EffectiveRefractiveIndex(n1,angle1,pol)
    if(pol == Pol.s):
        gamma = 1
    else:
        gamma = cos(angle0)/cos(angle1)
    mat = identity(2)
    for layer in layers:
        mat = dot(mat,layer.getSpecificMatrix(n0,angle0,wl,pol))  
    r = (eta0*(mat[0,0]+eta1*mat[0,1])-(mat[1,0]+eta1*mat[1,1]))/(eta0*(mat[0,0]+eta1*mat[0,1])+(mat[1,0]+eta1*mat[1,1]))
    t = gamma*2*eta0/(eta0*(mat[0,0]+eta1*mat[0,1])+(mat[1,0]+eta1*mat[1,1]))
    rr = abs(r)**2
    tt = n1*cos(angle1)/(n0*cos(angle0))*abs(t)**2
    return rr,tt

## マトリクス法
\begin{equation*}
 \left(\begin{array}{c}
  B \\
  C \\
 \end{array}\right)
 =
 \prod_{i=1}^{N}M_{i}
 \left(\begin{array}{c}
  1 \\
  \eta_{s} \\
 \end{array}\right)
\end{equation*}

In [12]:
def Admittance(matIn,matOut,layers,angle0,wl,pol):
    n0 = matIn.getIndex(wl)
    n1 = matOut.getIndex(wl)
    angle1 = SnellLaw(n0,n1,angle0)
    mat = identity(2)
    for layer in  layers:
        mat = dot(mat,layer.getSpecificMatrix(n0,angle0,wl,pol))
    eta1 = EffectiveRefractiveIndex(n1,angle1,pol)
    return dot(mat,array([[1],[eta1]]))

## 反射率
\begin{equation*}
 R = \left|\frac{B\eta_{0}-C}{B\eta_{0}+C}\right|^{2}
\end{equation*}

In [13]:
def Reflectance(matIn,Sub,matOut,layers,angle0,wl,pol):
    if pol == Pol.sp:
        return (Reflectance(matIn,Sub,matOut,layers,angle0,wl,Pol.s)
                  +Reflectance(matIn,Sub,matOut,layers,angle0,wl,Pol.p))/2
    else:
        matSub = Sub.material
        dSub = Sub.thickness
        ad = Admittance(matIn,matSub,layers,angle0,wl,pol)
        B = ad[0][0]
        C = ad[1][0]
        n0 = matIn.getIndex(wl)
        n1 = matSub.getIndex(wl)
        angle1 = SnellLaw(n0,n1,angle0)
        eta0 = EffectiveRefractiveIndex(n0,angle0,pol)
        eta1 = EffectiveRefractiveIndex(n1,angle1,pol)
        R02 = abs((B*eta0-C)/(B*eta0+C))**2
        T02 = 4*eta0.real*eta1.real/(abs(B*eta0+C)**2)
        R23,T23 = SemiInfinite(matSub,matOut,angle1,wl,pol)
        layers.reverse()
        ad = Admittance(matSub,matIn,layers,angle1,wl,pol)
        layers.reverse()
        B = ad[0][0]
        C = ad[1][0]
        R20 = abs((B*eta1-C)/(B*eta1+C))**2
        T20 = 4*eta1.real*eta0.real/(abs(B*eta1+C)**2)
        return (R02+T20*T20*R23/(1-R20*R23*exp(-4*pi*n1.imag*dSub/(wl*cos(angle1))))).real

## 透過率
\begin{equation*}
 T = \frac{4\eta_{0}\Re(\eta_{1})}{|B\eta_{0}+C|^{2}}
\end{equation*}


In [14]:
def Transmittance(matIn,Sub,matOut,layers,angle0,wl,pol):
    if pol == Pol.sp:
        return (Transmittance(matIn,Sub,matOut,layers,angle0,wl,Pol.s)
                  +Transmittance(matIn,Sub,matOut,layers,angle0,wl,Pol.p))/2
    else:
        matSub = Sub.material
        dSub = Sub.thickness
        ad = Admittance(matIn,matSub,layers,angle0,wl,pol)
        B = ad[0][0]
        C = ad[1][0]
        n0 = matIn.getIndex(wl)
        n1 = matSub.getIndex(wl)
        angle1 = SnellLaw(n0,n1,angle0)
        eta0 = EffectiveRefractiveIndex(n0,angle0,pol)
        eta1 = EffectiveRefractiveIndex(n1,angle1,pol)
        T02 = 4*eta0.real*eta1.real/(abs(B*eta0+C)**2)
        R23, T23 = SemiInfinite(matSub,matOut,angle1,wl,pol)
        layers.reverse()
        ad = Admittance(matSub,matIn,layers,angle1,wl,pol)
        layers.reverse()
        B = ad[0][0]
        C = ad[1][0]
        R20 = abs((B*eta1-C)/(B*eta1+C))**2
        return (T23*T02/(1-R20*R23*exp(-4*pi*n1.imag*dSub/(wl*cos(angle1))))).real

In [15]:
air = Material({550:1.0})
sub = Material({550:1.52})
h_mat = Material({550:2.2})
l_mat = Material({550:1.5})
h_layer = Layer(h_mat,100)
l_layer = Layer(l_mat,100)

In [16]:
angle =5/180*pi
pol =Pol.sp
print(SemiInfinite(air,sub,angle,550,pol))
print(Monolayer(air,sub,Layer(sub,0),angle,550,pol))
print((Reflectance(air,Layer(sub,100000),sub,[Layer(sub,0)],angle,550,pol),Transmittance(air,Layer(sub,100000),sub,[Layer(sub,0)],angle,550,pol)))

(0.042580974421055642, 0.95741902557894443)
(0.042580974421055642, 0.95741902557894443)
(0.042580974421055642, 0.95741902557894443)


In [17]:
angle =20/180*pi
pol =Pol.s
print(Monolayer(air,sub,h_layer,angle,550,pol))
print((Reflectance(air,Layer(sub,100000),sub,[h_layer],angle,550,pol),Transmittance(air,Layer(sub,100000),sub,[h_layer],angle,550,pol)))

(0.16045300254244507, 0.83954699745755501)
(0.16045300254244502, 0.8395469974575549)


In [18]:
angle =20/180*pi
pol =Pol.sp
layers=[l_layer,h_layer,l_layer,h_layer]
print(Reflectance(air,Layer(sub,100000),sub,layers,angle,550,pol))
print(Transmittance(air,Layer(sub,100000),sub,layers,angle,550,pol))

0.148047963161
0.851952036839


In [19]:
angle =20/180*pi
pol =Pol.sp
layers=[l_layer,h_layer,l_layer,h_layer]
print(Reflectance(air,Layer(sub,100000),air,layers,angle,550,pol))
print(Transmittance(air,Layer(sub,100000),air,layers,angle,550,pol))

0.179177985693
0.820822014307
